# Audio Data Processing
### Goal: take a .wav file as input => get a list of features corresponding to emotion dataset
(I want to make more data using existing files that matches the features I trained the emotion detecting model on and see how it does)

--- 

feature names - for each feature they computed the mean of the mean, mean of the std, std of the mean, and std of the std. I am assuming the inner computation is for that window size.

For example: 
- Mean_Acc1298_Mean_Mem40_Centroid numeric 
- Mean_Acc1298_Std_Mem40_Centroid numeric
- Std_Acc1298_Mean_Mem40_Centroid numeric
- Std_Acc1298_Std_Mem40_Centroid numeric


They do this for the following features:
1. Centroid (assuming this is the spectral centroid)
2. Rolloff
3. Flux
4. MFCC constants 1-12

I hope to do this and then plot the distributions to make sure that they fall into a similar distribution/range as the training data did (incase im accidentally scaling/not scaling something etc.)
other assumptions: i'm assuming the Mem40 means that the window size for the inner aggregation is 40.

In [1]:
#imports
import librosa
import numpy as np

#windowsize
mem_size = 40

In [2]:
feature_names = ['Centroid', 'Rolloff', 'Flux']
for i in range(0,13):
    feature_names.append("MFCC_"+str(i))
print(feature_names)

['Centroid', 'Rolloff', 'Flux', 'MFCC_0', 'MFCC_1', 'MFCC_2', 'MFCC_3', 'MFCC_4', 'MFCC_5', 'MFCC_6', 'MFCC_7', 'MFCC_8', 'MFCC_9', 'MFCC_10', 'MFCC_11', 'MFCC_12']


In [31]:
#centroid
y, sr = librosa.load("./data/movie_music/1.wav")
centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
rolloffs = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)[0]
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
S = np.abs(librosa.stft(y))
S_norm = librosa.util.normalize(S, axis=0)
flux = np.sqrt(np.sum(np.diff(S_norm, axis=1)**2, axis=0))

In [32]:
print(centroids.shape)
print(rolloffs.shape)
print(mfccs[i].shape)
print(mfccs.shape)
print(flux.shape)

(1445,)
(1445,)
(1445,)
(13, 1445)
(1444,)


In [35]:
#create the features
def compute_feature_aggregations(feature_list,feature_name):
    feature_array = np.array(feature_list)

    chunks = [
        feature_array[i:i+mem_size]
        for i in range(0, len(feature_array) - mem_size + 1, mem_size)
    ]

    chunk_means = [np.mean(chunk) for chunk in chunks]
    chunk_stds = [np.std(chunk) for chunk in chunks]

    results = {
        "Mean_Acc1298_Mean_Mem40_"+feature_name:float(np.mean(chunk_means)) ,
        "Mean_Acc1298_Std_Mem40_"+feature_name: float( np.mean(chunk_stds)),
        "Std_Acc1298_Mean_Mem40_"+feature_name: float( np.std(chunk_means)),
        "Std_Acc1298_Std_Mem40_"+feature_name:  float( np.std(chunk_stds)),
    }

    return results

data = {
    **compute_feature_aggregations(centroids, "Centroid"),
    **compute_feature_aggregations(rolloffs, "Rolloff"),
    **compute_feature_aggregations(flux, "Flux"),
}

for i in range(13):
    name = "MFCC_"+str(i)
    data = {**compute_feature_aggregations(mfccs[i],name), **data}
print(data)
    

{'Mean_Acc1298_Mean_Mem40_MFCC_12': -7.353248596191406, 'Mean_Acc1298_Std_Mem40_MFCC_12': 5.891010284423828, 'Std_Acc1298_Mean_Mem40_MFCC_12': 4.079969882965088, 'Std_Acc1298_Std_Mem40_MFCC_12': 1.3373233079910278, 'Mean_Acc1298_Mean_Mem40_MFCC_11': 2.949770927429199, 'Mean_Acc1298_Std_Mem40_MFCC_11': 5.783247947692871, 'Std_Acc1298_Mean_Mem40_MFCC_11': 5.9295268058776855, 'Std_Acc1298_Std_Mem40_MFCC_11': 1.0910663604736328, 'Mean_Acc1298_Mean_Mem40_MFCC_10': 6.195117473602295, 'Mean_Acc1298_Std_Mem40_MFCC_10': 5.600480079650879, 'Std_Acc1298_Mean_Mem40_MFCC_10': 3.6627042293548584, 'Std_Acc1298_Std_Mem40_MFCC_10': 0.9604876637458801, 'Mean_Acc1298_Mean_Mem40_MFCC_9': 11.387118339538574, 'Mean_Acc1298_Std_Mem40_MFCC_9': 6.32328987121582, 'Std_Acc1298_Mean_Mem40_MFCC_9': 3.769984722137451, 'Std_Acc1298_Std_Mem40_MFCC_9': 1.074137806892395, 'Mean_Acc1298_Mean_Mem40_MFCC_8': -2.333993911743164, 'Mean_Acc1298_Std_Mem40_MFCC_8': 5.693330764770508, 'Std_Acc1298_Mean_Mem40_MFCC_8': 2.66748666

In [46]:
#put it all together in a function
def compute_feature_aggregations(feature_list,feature_name):
    feature_array = np.array(feature_list)

    chunks = [
        feature_array[i:i+mem_size]
        for i in range(0, len(feature_array) - mem_size + 1, mem_size)
    ]

    chunk_means = [np.mean(chunk) for chunk in chunks]
    chunk_stds = [np.std(chunk) for chunk in chunks]

    results = {
        "Mean_Acc1298_Mean_Mem40_"+feature_name:float(np.mean(chunk_means)) ,
        "Mean_Acc1298_Std_Mem40_"+feature_name: float( np.mean(chunk_stds)),
        "Std_Acc1298_Mean_Mem40_"+feature_name: float( np.std(chunk_means)),
        "Std_Acc1298_Std_Mem40_"+feature_name:  float( np.std(chunk_stds)),
    }

    return results

def extract_features_from_wav(filename):
    y, sr = librosa.load(filename)
    centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    rolloffs = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)[0]
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    S = np.abs(librosa.stft(y))
    S_norm = librosa.util.normalize(S, axis=0)
    flux = np.sqrt(np.sum(np.diff(S_norm, axis=1)**2, axis=0))

  
    data = {
        **compute_feature_aggregations(centroids, "Centroid"),
        **compute_feature_aggregations(rolloffs, "Rolloff"),
        **compute_feature_aggregations(flux, "Flux"),
    }

    for i in range(13):
        name = "MFCC_"+str(i)
        data = {**compute_feature_aggregations(mfccs[i],name), **data}
    return data

#also do it for a bunch of files in a folder
from os import listdir
import pandas as pd
def audio_directory_to_df(audio_dir):
    files = [audio_dir+f for f in listdir(audio_dir) if f.endswith('.wav')]
    data = []
    for f in files:
        entry = extract_features_from_wav(f)
        print('entry', entry)
        data.append(entry)
    df = pd.DataFrame(data)
    return df

In [47]:
df=audio_directory_to_df('./data/instrument/musicnet/test_data/')
df.head()

entry {'Mean_Acc1298_Mean_Mem40_MFCC_12': 1.7178864479064941, 'Mean_Acc1298_Std_Mem40_MFCC_12': 9.827703475952148, 'Std_Acc1298_Mean_Mem40_MFCC_12': 12.644133567810059, 'Std_Acc1298_Std_Mem40_MFCC_12': 5.49846076965332, 'Mean_Acc1298_Mean_Mem40_MFCC_11': -7.366283893585205, 'Mean_Acc1298_Std_Mem40_MFCC_11': 9.013425827026367, 'Std_Acc1298_Mean_Mem40_MFCC_11': 9.73238754272461, 'Std_Acc1298_Std_Mem40_MFCC_11': 5.20632266998291, 'Mean_Acc1298_Mean_Mem40_MFCC_10': -5.940506935119629, 'Mean_Acc1298_Std_Mem40_MFCC_10': 8.913921356201172, 'Std_Acc1298_Mean_Mem40_MFCC_10': 9.795151710510254, 'Std_Acc1298_Std_Mem40_MFCC_10': 5.2408447265625, 'Mean_Acc1298_Mean_Mem40_MFCC_9': -4.015596389770508, 'Mean_Acc1298_Std_Mem40_MFCC_9': 8.56383991241455, 'Std_Acc1298_Mean_Mem40_MFCC_9': 11.397374153137207, 'Std_Acc1298_Std_Mem40_MFCC_9': 4.672379016876221, 'Mean_Acc1298_Mean_Mem40_MFCC_8': -20.476743698120117, 'Mean_Acc1298_Std_Mem40_MFCC_8': 8.17479419708252, 'Std_Acc1298_Mean_Mem40_MFCC_8': 11.0520143

,Mean_Acc1298_Mean_Mem40_MFCC_12,Mean_Acc1298_Std_Mem40_MFCC_12,Std_Acc1298_Mean_Mem40_MFCC_12,Std_Acc1298_Std_Mem40_MFCC_12,Mean_Acc1298_Mean_Mem40_MFCC_11,Mean_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Mean_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_11,Mean_Acc1298_Mean_Mem40_MFCC_10,Mean_Acc1298_Std_Mem40_MFCC_10,...,Std_Acc1298_Mean_Mem40_Centroid,Std_Acc1298_Std_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Std_Mem40_Rolloff,Std_Acc1298_Mean_Mem40_Rolloff,Std_Acc1298_Std_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Std_Mem40_Flux,Std_Acc1298_Mean_Mem40_Flux,Std_Acc1298_Std_Mem40_Flux
0,1.717886,9.827703,12.644134,5.498461,-7.366284,9.013426,9.732388,5.206323,-5.940507,8.913921,...,216.830481,130.161930,2151.228412,529.615068,489.760684,297.405120,0.612694,0.360522,0.139199,0.084081
1,-9.821401,7.468201,7.454530,3.120225,-13.509527,7.395432,6.503845,3.393321,-17.550457,7.192713,...,222.037955,134.336298,1403.115166,382.782211,480.228661,286.284007,0.734290,0.383194,0.181331,0.176073
2,-3.014821,7.453034,6.996465,3.510709,-3.737042,7.351256,7.363664,3.240184,-1.495759,6.759137,...,125.167981,42.932681,1300.706066,202.732684,335.157754,179.322709,0.632844,0.389242,0.174169,0.118699
3,-7.870029,4.638127,2.907183,1.092595,-3.878347,4.334676,2.845881,0.954173,0.238067,4.419583,...,210.865270,93.705872,1482.817150,401.642920,430.816727,165.774676,0.723453,0.386387,0.154986,0.268649
4,-5.108278,7.226155,7.145627,3.139133,-3.830916,7.478778,9.317322,2.964620,-10.407490,6.748012,...,282.795853,133.461334,2477.476652,455.574479,505.590920,263.386096,0.825114,0.385751,0.190428,0.209325
